# ESM vs WOA

#### Dask Import

In [1]:
from dask.distributed import Client

client = Client("tcp://127.0.0.1:33383")
client

Client Scheduler: tcp://127.0.0.1:33383 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 7 Cores: 7 Memory: 45.10 GB


<br>

#### Choices for data

In [2]:
#Experiment data for analysis
dataVariableId = 'thetao'
dataExperimentId = 'historical'
dataSourceID = 'E3SM-1-0'

#Custom Variables
latSel = slice(-89.5,-29.5) #Selected latitude to be investigated

<br>

#### Importing Libaries

In [3]:
import cartopy.crs as ccrs
import dask.dataframe as dd
import fsspec
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import xarray as xr
import zarr

from dask import delayed
from matplotlib.pyplot import cm
print("Imports complete")

Imports complete


<br>

#### Data Loading

<b>WOA Data</b>

In [4]:
dsWOA = pd.read_csv("woa18_decav_t00mn01.csv", skiprows =1,index_col=[0,1])
dsWOA

AND VALUES AT DEPTHS (M):0      5  \
COMMA SEPARATED LATITUDE  LONGITUDE                                       
-77.5                    -178.5                           -0.896 -0.922   
                         -177.5                           -0.720 -0.748   
                         -176.5                           -0.901 -0.925   
                         -175.5                           -0.802 -0.820   
                         -174.5                           -1.046 -1.084   
...                                                          ...    ...   
 89.5                     174.5                              NaN    NaN   
                          175.5                              NaN    NaN   
                          176.5                              NaN    NaN   
                          177.5                              NaN    NaN   
                          178.5                              NaN    NaN   

                                        10     15     20     25     30     35  \
COMMA SEPARATED LATITUDE  LONGITUDE                                             
-77.5                    -178.5     -0.942 -0.976 -1.001 -1.046 -1.094 -1.142   
                         -177.5     -0.763 -0.804 -0.854 -0.905 -0.990 -1.054   
                         -176.5     -0.996 -1.088 -1.132 -1.154 -1.215 -1.287   
                         -175.5     -0.839 -0.873 -0.926 -0.962 -0.965 -1.024   
                         -174.5     -1.108 -1.204 -1.287 -1.332 -1.346 -1.374   
...                                    ...    ...    ...    ...    ...    ...   
 89.5                     174.5     -1.695 -1.695 -1.696 -1.696 -1.697 -1.688   
                          175.5     -1.695 -1.696 -1.699 -1.699 -1.699 -1.699   
                          176.5     -1.702 -1.702 -1.702 -1.702 -1.699 -1.674   
                          177.5     -1.719 -1.718 -1.718 -1.718 -1.718 -1.718   
                          178.5     -1.729 -1.729 -1.729 -1.729 -1.729 -1.729   

                                        40     45  ...  4600  4700  4800  \
COMMA SEPARATED LATITUDE  LONGITUDE                ...                     
-77.5                    -178.5     -1.202 -1.277  ...   NaN   NaN   NaN   
                         -177.5     -1.111 -1.154  ...   NaN   NaN   NaN   
                         -176.5     -1.345 -1.382  ...   NaN   NaN   NaN   
                         -175.5     -1.057 -1.065  ...   NaN   NaN   NaN   
                         -174.5     -1.447 -1.467  ...   NaN   NaN   NaN   
...                                    ...    ...  ...   ...   ...   ...   
 89.5                     174.5     -1.638 -1.618  ...   NaN   NaN   NaN   
                          175.5     -1.663 -1.638  ...   NaN   NaN   NaN   
                          176.5     -1.613 -1.554  ...   NaN   NaN   NaN   
                          177.5     -1.708 -1.701  ...   NaN   NaN   NaN   
                          178.5     -1.645 -1.620  ...   NaN   NaN   NaN   

                                     4900  5000  5100  5200  5300  5400  5500  
COMMA SEPARATED LATITUDE  LONGITUDE                                            
-77.5                    -178.5       NaN   NaN   NaN   NaN   NaN   NaN   NaN  
                         -177.5       NaN   NaN   NaN   NaN   NaN   NaN   NaN  
                         -176.5       NaN   NaN   NaN   NaN   NaN   NaN   NaN  
                         -175.5       NaN   NaN   NaN   NaN   NaN   NaN   NaN  
                         -174.5       NaN   NaN   NaN   NaN   NaN   NaN   NaN  
...                                   ...   ...   ...   ...   ...   ...   ...  
 89.5                     174.5       NaN   NaN   NaN   NaN   NaN   NaN   NaN  
                          175.5       NaN   NaN   NaN   NaN   NaN   NaN   NaN  
                          176.5       NaN   NaN   NaN   NaN   NaN   NaN   NaN  
                          177.5       NaN   NaN   NaN   NaN   NaN   NaN   NaN  
                          178.5       NaN   NaN   NaN   NaN

In [5]:
dfWOA = dsWOA.to_xarray()
dfWOA = dfWOA.rename({'COMMA SEPARATED LATITUDE':'LATITUDE'})
dfWOA

<xarray.Dataset>
Dimensions:                      ( LONGITUDE: 360, LATITUDE: 168)
Coordinates:
  * LATITUDE                     (LATITUDE) float64 -77.5 -76.5 ... 88.5 89.5
  *  LONGITUDE                   ( LONGITUDE) float64 -179.5 -178.5 ... 179.5
Data variables:
     AND VALUES AT DEPTHS (M):0  (LATITUDE,  LONGITUDE) float64 nan ... nan
    5                            (LATITUDE,  LONGITUDE) float64 nan ... nan
    10                           (LATITUDE,  LONGITUDE) float64 nan ... nan
    15                           (LATITUDE,  LONGITUDE) float64 nan ... nan
    20                           (LATITUDE,  LONGITUDE) float64 nan ... nan
    25                           (LATITUDE,  LONGITUDE) float64 nan ... nan
    30                           (LATITUDE,  LONGITUDE) float64 nan ... nan
    35                           (LATITUDE,  LONGITUDE) float64 nan ... nan
    40                           (LATITUDE,  LONGITUDE) float64 nan ... nan
    45                           (LATITUDE,  LONGITUDE) float64 nan ... nan
    50                           (LATITUDE,  LONGITUDE) float64 nan ... nan
    55                           (LATITUDE,  LONGITUDE) float64 nan ... nan
    60                           (LATITUDE,  LONGITUDE) float64 nan ... nan
    65                           (LATITUDE,  LONGITUDE) float64 nan ... nan
    70                           (LATITUDE,  LONGITUDE) float64 nan ... nan
    75                           (LATITUDE,  LONGITUDE) float64 nan ... nan
    80                           (LATITUDE,  LONGITUDE) float64 nan ... nan
    85                           (LATITUDE,  LONGITUDE) float64 nan ... nan
    90                           (LATITUDE,  LONGITUDE) float64 nan ... nan
    95                           (LATITUDE,  LONGITUDE) float64 nan ... nan
    100                          (LATITUDE,  LONGITUDE) float64 nan ... nan
    125                          (LATITUDE,  LONGITUDE) float64 nan ... nan
    150                          (LATITUDE,  LONGITUDE) float64 nan ... nan
    175                          (LATITUDE,  LONGITUDE) float64 nan ... nan
    200                          (LATITUDE,  LONGITUDE) float64 nan ... nan
    225                          (LATITUDE,  LONGITUDE) float64 nan ... nan
    250                          (LATITUDE,  LONGITUDE) float64 nan ... nan
    275                          (LATITUDE,  LONGITUDE) float64 nan ... nan
    300                          (LATITUDE,  LONGITUDE) float64 nan ... nan
    325                          (LATITUDE,  LONGITUDE) float64 nan ... nan
    350                          (LATITUDE,  LONGITUDE) float64 nan ... nan
    375                          (LATITUDE,  LONGITUDE) float64 nan ... nan
    400                          (LATITUDE,  LONGITUDE) float64 nan ... nan
    425                          (LATITUDE,  LONGITUDE) float64 nan ... nan
    450                          (LATITUDE,  LONGITUDE) float64 nan ... nan
    475                          (LATITUDE,  LONGITUDE) float64 nan ... nan
    500                          (LATITUDE,  LONGITUDE) float64 nan ... nan
    550                          (LATITUDE,  LONGITUDE) float64 nan ... nan
    600                          (LATITUDE,  LONGITUDE) float64 nan ... nan
    650                          (LATITUDE,  LONGITUDE) float64 nan nan ... nan
    700                          (LATITUDE,  LONGITUDE) float64 nan nan ... nan
    750                          (LATITUDE,  LONGITUDE) float64 nan nan ... nan
    800                          (LATITUDE,  LONGITUDE) float64 nan nan ... nan
    850                          (LATITUDE,  LONGITUDE) float64 nan nan ... nan
    900                          (LATITUDE,  LONGITUDE) float64 nan nan ... nan
    950                          (LATITUDE,  LONGITUDE) float64 nan nan ... nan
    1000                         (LATITUDE,  LONGITUDE) float64 nan nan ... nan
    1050                         (LATITUDE,  LONGITUDE) float64 nan nan ... nan
    1100                         (LATITUDE,  LON

<br>
<b>ESM Data</b>

In [6]:
df = pd.read_csv('https://storage.googleapis.com/cmip6/cmip6-zarr-consolidated-stores.csv')
dfFilt = df[df.variable_id.eq('thetao') & df.experiment_id.eq('historical') & df.source_id.eq('E3SM-1-0')]
dfArea = df[df.variable_id.eq('areacello') & df.source_id.eq('CESM2')]

fileSetList = []
for i in range(len(dfFilt)):
    zstore = dfFilt.zstore.values[i]
    mapper = fsspec.get_mapper(zstore)
    fileRaw = xr.open_zarr(mapper, consolidated=True)
    fileSetList.append(fileRaw)
fileCount = len(fileSetList)

dsArea = xr.open_zarr(fsspec.get_mapper(dfArea.zstore.values[1]), consolidated=True)
dataAreaRaw = dsArea.areacello

for i in range(fileCount): #Formatting dates into np.datetime64 format
    startDateIterate = np.datetime64(fileSetList[i]['time'].values[0],'M')
    endDateIterate = np.datetime64(fileSetList[i]['time'].values[-1],'M') + np.timedelta64(1,'M')
    fileSetList[i]['time']=('time', np.arange(startDateIterate, endDateIterate, dtype='datetime64[M]'))
    fileSetList[i]['time_bnds']=('time_bnds', np.arange(startDateIterate, endDateIterate, dtype='datetime64[M]')) 
fileSet = xr.combine_nested(fileSetList, concat_dim='SetId') #Combining data sets

dataFiltLatLev = fileSet.thetao.sel(lat=latSel) #Selection of latitude and depth
dataFiltFull = dataFiltLatLev.reset_coords(drop=True) #Removes lev if single value

globalStartDate = dataFiltFull["time"][0].values
globalDateInc = dataFiltFull["time"][1].values - globalStartDate
#np.datetime64(globalDateInc,'D')
globalEndDateIn = dataFiltFull["time"][-1].values
globalEndDateOut = globalEndDateIn + globalDateInc

globalStartDateStr = str(globalStartDate)[:7]
globalEndDateInStr = str(globalEndDateIn)[:7]
globalEndDateOutStr = str(globalEndDateOut)[:7]

print("Data loaded")

Data loaded
